In [2]:
from aoc import *
from copy import deepcopy
from collections import defaultdict, Counter, deque
import re
from z3 import Ints, Solver, sat
from tqdm import tqdm

year = 2024
day = 20

download_input(year, day)

In [3]:
aoc, lines, G, R, C = read_input(day, test=True)

['###############', '#...#...#.....#', '#.#.#.#.#.###.#', '#S#...#.#.#...#', '#######.#.#.###']
15 15


In [4]:
for r, row in enumerate(G):
    for c, cell in enumerate(row):
        if cell == 'S':
            s = (r,c)
            G[r][c] = '.'
        if cell == 'E':
            e = (r,c)
            G[r][c] = '.'

print(s,e)

(3, 1) (7, 5)


In [5]:
Q = deque([(s, 0)])
SEEN = set()
shortest_length = 0

while Q:
    pos, cost = Q.popleft()
    r, c = pos

    if (r,c) == e:
        shortest_length = cost

    if (r, c) in SEEN:
        continue
    else:
        SEEN.add((r,c))

    for d in dirs:
        rr = r + d[0]
        cc = c + d[1]
        if 0 <= rr < len(G) and 0 <= cc < len(G[rr]):
            if G[rr][cc] == ".":
                Q.append(((rr,cc), cost+1))


print(f"{shortest_length}=")


Q = [(0, s, ())]


import heapq

SEEN = set()

answers = Counter()

while Q:
    cost, pos, cheat = heapq.heappop(Q)
    r, c = pos

    if cost > shortest_length - 100:
        continue

    if (r,c) == e:
        if cost < shortest_length:
            answers[shortest_length - cost] += 1

    if (r, c, cheat) in SEEN:
        continue
    else:
        SEEN.add((r,c, cheat))

    for d in dirs:
        rr = r + d[0]
        cc = c + d[1]
        if 0 <= rr < len(G) and 0 <= cc < len(G[0]):
            if len(cheat) == 1:
                heapq.heappush(Q,(cost+1, (rr,cc), cheat[0] + (rr,cc)))
            elif G[rr][cc] == "#" and len(cheat) == 0:
                heapq.heappush(Q,(cost+1, (rr,cc), (rr,cc)))
            elif G[rr][cc] == ".":
                heapq.heappush(Q,(cost+1, (rr,cc), cheat))
            else:
                pass

from pprint import pprint
pprint(answers)

print(sum([v for k,v in answers.items() if k < shortest_length]))

84=
Counter()
0


In [ ]:
import heapq

f = (0, 1)


def next_move(search):
    cost, state = search
    r, c, f, path = state

    if (r, c, f) in SEEN:
        return []
    else:
        SEEN.add((r, c, f))

    if (r, c) == e:
        paths.append(search)
        return []
    if (r, c, f) in path:
        return []

    path = tuple([x for x in path] + [(r, c, f)])

    available_steps = []

    for d in dirs:
        rr = r + d[0]
        cc = c + d[1]

        if f != d:
            # don't turn to face a wall.
            if G[rr][cc] == ".":
                available_steps.append((cost + 1000, (r, c, d, path)))
        else:
            if G[rr][cc] == ".":
                available_steps.append((cost + 1, (rr, cc, f, path)))

    return available_steps


start_state = (s[0], s[1], f, ())

Q = [(0, start_state)]
SEEN = set()
paths = []

while Q:
    this_move = heapq.heappop(Q)
    next_moves = next_move(this_move)
    for n in next_moves:
        heapq.heappush(Q, n)
    if len(paths) > 0:
        break

print(paths[0][0])

In [30]:
Q = deque([(s,  0)])
SEEN = set()

shortest_length = 0

while Q:
    pos, cost = Q.popleft()
    r, c = pos

    if (r,c) == e:
        shortest_length = cost

    if (r, c) in SEEN:
        continue
    else:
        SEEN.add((r,c))

    for d in dirs:
        rr = r + d[0]
        cc = c + d[1]
        if 0 <= rr < len(G) and 0 <= cc < len(G[rr]):
            if G[rr][cc] == ".":
                Q.append(((rr,cc), cost+1))

pairs_to_try = set()

for r, row in enumerate(G):
    for c, cell in enumerate(row):
        for d in dirs:
            rr = r + d[0]
            cc = c + d[1]
            if 0 <= rr < len(G) and 0 <= cc < len(G[0]):
                # if G[rr][cc] == "#" or G[r][c] =='#':
                #     pairs_to_try.add(frozenset({(r,c),(rr,cc)}))
                if G[r][c] =='#':
                    pairs_to_try.add(frozenset({(r,c)}))

def perform_race(ignored_locations):

    Q = deque([(s, 0, 0)])
    SEEN = set()

    while Q:
        pos, cheat, cost = Q.popleft()
        r, c = pos

        if cost + MINIMUM_SAVING > shortest_length:
            return 1e9

        if (r,c) == e:
            if cheat == len(ignored_locations):
                return cost
            else:
                return 1e9

        if (r, c, cheat) in SEEN:
            continue
        else:
            SEEN.add((r,c, cheat))

        # if len(cheat) > 0:
        #     print(cheat)
        #     print(SEEN)
        #     return

        for d in dirs:
            rr = r + d[0]
            cc = c + d[1]
            if 0 <= rr < len(G) and 0 <= cc < len(G[0]):
                if (rr, cc) in ignored_locations:
                    Q.append(((rr,cc), cheat + 1, cost+1))
                elif G[rr][cc] == ".":
                    Q.append(((rr,cc), cheat, cost+1))

answers = Counter()

print(f"{len(pairs_to_try)=}")

aoc, lines, G, R, C = read_input(day, test=True)
for r, row in enumerate(G):
    for c, cell in enumerate(row):
        if cell == 'S':
            s = (r, c)
            G[r][c] = '.'
        if cell == 'E':
            e = (r, c)
            G[r][c] = '.'



MINIMUM_SAVING = 100

for pair in tqdm(pairs_to_try):

    this_cost = perform_race(pair)
    if shortest_length - this_cost > 0:
        answers[shortest_length - this_cost] += 1


from pprint import pprint
pprint(answers)
#pprint(pairs_to_try)

print(sum([v for k,v in answers.items()]))


len(pairs_to_try)=10424
['###############', '#...#...#.....#', '#.#.#.#.#.###.#', '#S#...#.#.#...#', '#######.#.#.###']
15 15


100%|██████████| 10424/10424 [00:00<00:00, 11444.76it/s]

Counter({9372: 25,
         9374: 14,
         9376: 6,
         9382: 2,
         9378: 2,
         9384: 2,
         9412: 1,
         9380: 1,
         9410: 1,
         9436: 1})
55


In [18]:
Q = deque([(s, 0)])
COST_GRID = [[0 for _ in range(len(G[0]))] for _ in range(len(G))]
COST_GRID_STR = [['   ' for _ in range(len(G[0]))] for _ in range(len(G))]
SEEN = set()


while Q:
    pos, cost = Q.popleft()
    r, c = pos

    # if (r,c) == e:
    #     shortest_length = cost

    if (r, c) in SEEN:
        continue
    else:
        SEEN.add((r,c))
        COST_GRID[r][c] = cost
        COST_GRID_STR[r][c] = str(cost).zfill(3)

    for d in dirs:
        rr = r + d[0]
        cc = c + d[1]
        if 0 <= rr < len(G) and 0 <= cc < len(G[rr]):
            if G[rr][cc] == ".":
                Q.append(((rr,cc), cost+1))

#print(COST_GRID_STR)
print_grid(COST_GRID_STR)

                                             
   002003004   010011012   026027028029030   
   001   005   009   013   025         031   
   000   006007008   014   024   034033032   
                     015   023   035         
                     016   022   036037038   
                     017   021         039   
         082083084   018019020   042041040   
         081                     043         
   078079080         060059058   044045046   
   077               061   057         047   
   076   070069068   062   056   050049048   
   075   071   067   063   055   051         
   074073072   066065064   054053052         
                                             


